In [148]:
import pandas as pd
import numpy as np
import os.path
from pandas_ods_reader import read_ods

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers

import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

%matplotlib inline

In [149]:
fighter_sheet = "raw_fighter_details"
fight_sheet = "raw_total_fight_data"


fighters_ods = "UFC-DATA-DELETE-LATER/raw_fighter_details.ods"
fights_ods = "UFC-DATA-DELETE-LATER/raw_total_fight_data.ods"


In [183]:
fights_data = read_ods(fights_ods, fight_sheet)
fighters_data = read_ods(fighters_ods, fighter_sheet)

In [184]:
fights_data['R_SIG_STR_pct'] = fights_data['R_SIG_STR_pct'].str.strip('%').astype(float)
fights_data['B_SIG_STR_pct'] = fights_data['B_SIG_STR_pct'].str.strip('%').astype(float)
fights_data['R_TD_pct'] = fights_data['R_TD_pct'].str.strip('%').astype(float)
fights_data['B_TD_pct'] = fights_data['B_TD_pct'].str.strip('%').astype(float)

In [185]:
fights_data['R_SIG_STR_pct'] = fights_data['R_SIG_STR_pct']/100
fights_data['B_SIG_STR_pct'] = fights_data['B_SIG_STR_pct']/100
fights_data['R_TD_pct'] = fights_data['R_TD_pct']/100
fights_data['B_TD_pct'] = fights_data['B_TD_pct']/100


In [186]:
fights_data.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,R_TD,B_TD,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,R_PASS,B_PASS,R_REV,B_REV,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_DISTANCE,B_DISTANCE,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Henry Cejudo,Marlon Moraes,0.0,0.0,90 of 171,57 of 119,0.52,0.47,99 of 182,59 of 121,1 of 4,0 of 2,0.25,0.00,1.0,0.0,1.0,0.0,0.0,0.0,73 of 150,35 of 89,13 of 16,7 of 8,4 of 5,15 of 22,45 of 118,54 of 116,19 of 23,2 of 2,26 of 30,1 of 1,KO/TKO,3.0,4:51,5 Rnd (5-5-5-5-5),Marc Goddard,"June 08, 2019","Chicago, Illinois, USA",UFC Bantamweight Title Bout,Henry Cejudo
1,Valentina Shevchenko,Jessica Eye,1.0,0.0,8 of 11,2 of 12,0.72,0.16,37 of 40,42 of 52,2 of 2,0 of 0,1.00,0.00,1.0,0.0,3.0,0.0,0.0,0.0,4 of 5,0 of 7,4 of 6,0 of 2,0 of 0,2 of 3,5 of 8,2 of 12,2 of 2,0 of 0,1 of 1,0 of 0,KO/TKO,2.0,0:26,5 Rnd (5-5-5-5-5),Robert Madrigal,"June 08, 2019","Chicago, Illinois, USA",UFC Women's Flyweight Title Bout,Valentina Shevchenko
2,Tony Ferguson,Donald Cerrone,0.0,0.0,104 of 200,68 of 185,0.52,0.36,104 of 200,68 of 185,0 of 0,1 of 1,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,65 of 144,43 of 152,25 of 37,15 of 23,14 of 19,10 of 10,103 of 198,68 of 184,1 of 2,0 of 1,0 of 0,0 of 0,TKO - Doctor's Stoppage,2.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Lightweight Bout,Tony Ferguson
3,Jimmie Rivera,Petr Yan,0.0,2.0,73 of 192,56 of 189,0.38,0.29,76 of 195,58 of 192,0 of 3,1 of 3,0.00,0.33,0.0,0.0,0.0,1.0,0.0,0.0,42 of 145,40 of 166,15 of 24,13 of 19,16 of 23,3 of 4,60 of 173,42 of 167,9 of 15,10 of 12,4 of 4,4 of 10,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Kevin MacDonald,"June 08, 2019","Chicago, Illinois, USA",Bantamweight Bout,Petr Yan
4,Tai Tuivasa,Blagoy Ivanov,0.0,1.0,64 of 144,73 of 123,0.44,0.59,66 of 146,81 of 131,0 of 0,2 of 2,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,39 of 114,65 of 114,6 of 7,7 of 8,19 of 23,1 of 1,50 of 126,62 of 111,14 of 18,5 of 6,0 of 0,6 of 6,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Heavyweight Bout,Blagoy Ivanov


In [192]:
r_total_str = fights_data['R_TOTAL_STR.'].str.split(' of ', expand = True)
r_total_str['R_TOTAL_STR.'] = r_total_str[0].astype(float)/r_total_str[1].astype(float)

temp_df = fights_data['B_TOTAL_STR.'].str.split(' of ', expand = True)
temp_df['B_TOTAL_STR.'] = temp_df[0].astype(float)/temp_df[1].astype(float)

temp_df = fights_data['R_TD'].str.split(' of ', expand = True)
temp_df['R_TD'] = temp_df[0].astype(float)/temp_df[1].astype(float)

temp_df = fights_data['B_TD'].str.split(' of ', expand = True)
temp_df['B_TD'] = temp_df[0].astype(float)/temp_df[1].astype(float)

In [180]:
fights_data.head()

,0,1,R_TOTAL_STR.
0,99,182,0.543956
1,37,40,0.925000
2,104,200,0.520000
3,76,195,0.389744
4,66,146,0.452055


In [100]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [172]:
# Creating input feature
X = fights_data[['R_KD','B_KD','R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
                 'R_TD','B_TD','R_TD_pct','B_TD_pct','R_SUB_ATT','B_SUB_ATT','R_PASS','B_PASS','R_REV','B_REV','R_HEAD', 
                 'B_HEAD', 'R_BODY', 'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH', 'B_CLINCH',
                'R_GROUND', 'B_GROUND']]

# Creating target variable
y = fights_data[['Winner']]

#Creating Train and Test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Normalizing the input features
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.transform(X_test)

#Normalizing the target variable
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train)
y_test = sc_y.transform(y_test)

# Creating the Linear regressor
regressor = LinearRegression()

# fitting the training data to the Linear Regressor
regressor.fit(X_train,y_train)

ValueError: could not convert string to float: '147 of 199'